# Zasady budowania modeli deep learning

## Model

Gdy dane są już gotowe kolejnym krokiem jest odpowiednie zdefiniowanie modelu na którym będziemy pracować. Wykorzystamy do tego bibliotekę [flux.jl](http://fluxml.ai/):

- [Flux](http://fluxml.ai/) jest biblioteką Julii przeznaczoną do tworzenia modeli uczenia maszynowego.
- Jest w całości oparta na Julii, przez co trywialne jest jej modyfikowanie i dostosowywanie do swoich potrzeb. 
- Możliwe jest przy tym wykorzystanie wewnątrz modeli składni, funkcji i makr Julii.
- Przy czym tworzenie całkiem złożonych standardowych modeli jest intuicyjne i szybkie, zazwyczaj zajmują one jedynie kilka linijek.

### Warstwy sieci neuronowej

Jak już wspomnieliśmy wcześniej Flux jest wpełni modyfikowalny i możemy samodzielnie zdefiniować warstwy takiej sieci, korzystając np. z sigmoidalnej funkcją aktywacji:

In [1]:
using Flux

In [2]:
W = rand(4, 8)
b = rand(4)
layer₁(x) = 1.0 ./ (1.0.+exp.(-W*x - b))

layer₁ (generic function with 1 method)

In [3]:
x = rand(8)
layer₁(x)

4-element Array{Float64,1}:
 0.8833785291496846
 0.9705194475207206
 0.9090854473650531
 0.9536472632262277

W przypadku gdy chcemy wykorzystać narzędzia wbudowane we Fluxa (np. propagację wsteczną i automatyczne różniczkowanie) musimy zdefiniować warstwę jako [modyfikowalne parametry modelu:](https://github.com/FluxML/Tracker.jl/tree/master/src)

In [4]:
using Flux.Tracker

W = param(W)
b = param(b)

Tracked 4-element Array{Float64,1}:
 0.08604460009379467
 0.7533683005291176 
 0.3652667082489065 
 0.19352842362552325

Przy czym w przypaku najpowszechniejszych funkcji nie musimy ich samodzielnie deklarować. Flux dostarcza  najpopularniejsze funkcje aktywacji i podstawowe typy [warstw modelu](https://fluxml.ai/Flux.jl/stable/models/layers/#Basic-Layers-1):

In [5]:
layer₂(x) = σ.(W * x .+ b)
layer₂(x)

Tracked 4-element Array{Float64,1}:
 0.8833785291496846
 0.9705194475207206
 0.9090854473650531
 0.9536472632262277

In [6]:
layer₃ = Dense(8,4,σ)
layer₃(x)

Tracked 4-element Array{Float32,1}:
 0.14636862f0
 0.6760975f0 
 0.75517964f0
 0.7181234f0 

Możemy zdefiniować też własnne warstwy:

In [7]:
struct Poly
    W
    V
    b
end

Poly(in::Integer, out::Integer) =
  Poly(param(randn(out, in)),randn(out, in), param(randn(out)))

# Overload call, so the object can be used as a function
(m::Poly)(x) = m.W * x.^2 + m.V*x .+ m.b

a = Poly(10, 5)

a(rand(10)) # => 5-element vector

Tracked 5-element Array{Float64,1}:
  2.004568405454809 
 -3.8277976573511583
  2.374411373520858 
  4.977538072956816 
 -0.8848032694490391

Znów, samo zdefioniowanie warstwy jako obiektu nie wystarczy do wykorzystania wszystkich funkcji Fluxa. Gdy chcemy wykorzystać wbudowane we Fluxa narzędzia do wyznaczania gradientu czy też [liczyć model na GPU](https://fluxml.ai/Flux.jl/stable/gpu/)  musimy jeszcze skorzystać z makra <tt>treelike</tt>:

In [9]:
Flux.@treelike Poly

Chcąc zbudować model z więcej niż jedną warstwą musimy go odpowiednio zdefiniować:

In [10]:
Layer₁ = Dense(28^2, 32, relu)
Layer₂ = Dense(32, 10)
Layer₃ = softmax

softmax (generic function with 2 methods)

Funkcja <tt>Chain</tt> pozwala łączyć w łancuchy dowolne funkcje w Julii:

In [11]:
chain = Chain(x -> x^2, x-> -x)
m₁ = Chain(Layer₁ , Layer₂, Layer₃) 

Chain(Dense(784, 32, NNlib.relu), Dense(32, 10), NNlib.softmax)

Możemy zdefiniować model także jako złożenie funkcji:

In [12]:
m₂(x) = Layer₃(Layer₂(Layer₁(x)))

m₂ (generic function with 1 method)

In [13]:
m₃(x) = Layer₁ ∘ Layer₂ ∘ Layer₃  

m₃ (generic function with 1 method)

Albo jako potok:

In [14]:
m₄(x) = Layer₁(x) |> Layer₂  |> Layer₃ 

m₄ (generic function with 1 method)

### Funkcje straty i regularyzacja

[Goodfellow I., Bengio Y., Courville A. (2016), Deep Learning, rozdział 7](http://www.deeplearningbook.org/contents/regularization.html)

Tak jak mówiliśmy na poprzednim wykładzie nie mamy możliwości bezpośredniej optymalizacji wag $\theta$ w modelu. Do procesu uczenia musimy wykorzystać funkcję straty $J(\theta)$:


[![](https://ai2-s2-public.s3.amazonaws.com/figures/2017-08-08/d589392ab606a3d2861988ebcba95176517939ec/2-Table1-1.png)](https://www.semanticscholar.org/paper/On-Loss-Functions-for-Deep-Neural-Networks-in-Janocha-Czarnecki/d589392ab606a3d2861988ebcba95176517939ec)

Funkcję straty możemy zdefiniować samodzielnie:

In [15]:
model = Dense(5,2)
x, y = rand(5), rand(2);
loss(ŷ, y) = sum((ŷ.- y).^2)/ length(y)
loss(model(x), y) 

0.49787496735826986 (tracked)

albo wykorzystać [jedną z zaimplementowanych we Fluxie:](https://github.com/FluxML/Flux.jl/blob/8f73dc6e148eedd11463571a0a8215fd87e7e05b/src/layers/stateless.jl):

In [16]:
Flux.mse(model(x),y)

0.49787496735826986 (tracked)

Jednak samo zdefiniowanie funkcji straty nie wystarczy. Dobry model uczenia maszynowego musi mieć możliwie jak najniższy <b>błąd generalizacji</b>:

[![](https://cdn-images-1.medium.com/max/1600/1*1woqrqfRwmS1xXYHKPMUDw.png)](https://buzzrobot.com/bias-and-variance-11d8e1fee627)


Niestety sieci neuronowe mają tendencję do przeuczania się i w przypadku ich używania konieczne jest wykorzystanie odpowiedniej metody <b>regularyzacji</b>. Dzięki temu możliwe będzie zaproponowanie modelu, który będzie umiał efektywnie aproksymować dane inne niż trenujące.

Do najczęściej wykorzystywanych metod regularyzacji należą:


<b>nakładanie kar na parametry</b>:

Jeden z najczęściej wykorzystywanych sposobów regularyzacji. Polega on na nałożeniu unormowanej kary na parametry funkcji straty: 
     
$\tilde{J}(\theta) = J(\theta) + \alpha\Omega(\theta)$

Najczęściej spotykamy się z postaciami:
- $\Omega(\theta) = ||w||_1 = \sum_i{|w_i|}$     (<i>LASSO</i>,<i>regularyzacja $L_1$</i>)
- $\Omega(\theta) = ||w||_2^2 = \sum_i{|w_i|}$ (<i>regularyzacja Tichonowa</i>, <i>regresja grzbietowa</i>,<i>regularyzacja $L_2$</i>)

Ich implementacja wyglądałaby [następująco](https://fluxml.ai/Flux.jl/stable/models/regularisation/):

In [17]:
using LinearAlgebra

In [18]:
penalty() =  LinearAlgebra.norm(model.W,1) + LinearAlgebra.norm(model.b,1) #L1
penalty() =  0.5LinearAlgebra.norm(model.W,2) + LinearAlgebra.norm(model.b,2) #L2 lub:
penalty() =  LinearAlgebra.norm(model.W) + LinearAlgebra.norm(model.b) #L1

penalty (generic function with 1 method)

In [20]:
penalty()

1.1553358639951514 (tracked)

In [ ]:
loss(model(x),y) + penalty()

czy nawet prościej:

In [19]:
sum(LinearAlgebra.norm,params(model))

1.1553358639951514 (tracked)

<b>Bagging (bootstrap aggregating)</b>:

Polega on na losowaniu ze zwracaniem $k$ próbek z wejściowego zbioru danych i szacowaniu na nich $k$  modeli, a następnie uśrednianiu ich rezultatów.

<b>Dropout</b>:

Polega na tworzeniu nowych modeli poprzez usuwanie neuronów z warstw ukrytych z prawdopodobieństwem $p$ w każdej iteracji uczenia. Niech wektor $\mu = [1,1,0,1,1,1,\dots,0,1]$ oznacza neurony wykorzystane do uczenia modelu w danej iteracji $i$. W takim wypadku procedura uczenia sprowadza się do minimalizacji wartości wyrażenia $E_\mu[J(\theta,\mu)]$ dla każdej kolejnej iteracji. Dzięki temu otrzymujemy nieobciążony estymator gradientu bez konieczności generowania i uczenia $k$ modeli tak jak w przypadku baggingu.

Dropout implementuje się we Fluxie jako [warstwę modelu](https://fluxml.ai/Flux.jl/stable/models/layers/#Normalisation-and-Regularisation-1): 

In [23]:
model = Chain(Dense(28^2, 32, relu),
    Dropout(0.1),
Dense(32, 10),
BatchNorm(64, relu),
softmax)

Chain(Dense(784, 32, NNlib.relu), Dropout{Float64}(0.1, true), Dense(32, 10), BatchNorm(64, λ = NNlib.relu), NNlib.softmax)

### Optymalizacja sieci

[Goodfellow I., Bengio Y., Courville A. (2016), Deep Learning, rozdział 8](http://www.deeplearningbook.org/contents/optimization.html)

Dobór odpowiedniego algorytmu optymalizacyjnego jest jednym z najważniejszych kroków w trakcie przygotowywania sieci neuronowej. Specyfika procesu ich uczenia powoduje, że proces optymalizacji jest podatny na wiele potencjalnych problemów, między innymi:
- złe uwarunkowanie macierzy.
- występowanie lokalnych minimów, punktów siodłowych, etc.
- zjawisko zanikającego gradientu

Z tego powodu istnieje wiele różnych algorytmów, które próbują przeciwdziałać wymienionym powyżej problemom. To który z nich powinien być zastosowany zależy tak naprawdę od specyfiki rozpatrywanego przypadku. Do najpopularnieszych należą:
- SGD [(Robbins & Munro 1951)](https://projecteuclid.org/download/pdf_1/euclid.aoms/1177729586)
- SGD z pędem (momentum) [(Polyak, 1964)](http://www.mathnet.ru/php/archive.phtml?wshow=paper&jrnid=zvmmf&paperid=7713&option_lang=eng)
- SGD z pędem Nesterova ([Nesterov, 1983](http://www.cis.pku.edu.cn/faculty/vision/zlin/1983-A%20Method%20of%20Solving%20a%20Convex%20Programming%20Problem%20with%20Convergence%20Rate%20O%28k%5E%28-2%29%29_Nesterov.pdf), [2005](https://www.math.ucdavis.edu/~sqma/MAT258A_Files/Nesterov-2005.pdf))
- AdaGrad (Adaptive Gradient Algorithm) [(Duchi et. al. 2011)](http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf)
- ADAM (Adaptive Moment Estimation) [(Kingma & Ba, 2015)](https://arxiv.org/abs/1412.6980)



Flux umożliwia [samodzielne zdefiniowanie gradientu](https://fluxml.ai/Flux.jl/stable/internals/tracker/#Custom-Gradients-1) i przekazanie go do modelu:

In [24]:
minus(a, b) = a - b

minus (generic function with 1 method)

In [44]:
using Flux.Tracker: TrackedArray, track, data, @grad

minus(a::TrackedArray, b::TrackedArray) = track(minus, a, b)

minus (generic function with 2 methods)

In [45]:
@grad function minus(a, b)
  return minus(data(a), data(b)), Δ -> (Δ, -Δ)
end

In [46]:
a = param([1,2,3])
b = param([3,2,1])

c = minus(a, b)  # [-2.0 (tracked), 0.0 (tracked), 2.0 (tracked)]

Tracker.back!(c, 1)
Tracker.grad(a)  # [1.00, 1.00, 1.00]
Tracker.grad(b)  # [-1.00, -1.00, -1.00]

3-element Array{Float64,1}:
 -1.0
 -1.0
 -1.0

Najczęściej nie jest jednak konieczne samodzielne definiowanie gradientu, Flux umie samodzielnie wyliczyć pochodną dowolnej funkcji:

In [28]:
f(x) = 3x^2 + 2x + 1

# df/dx = 6x + 2
df(x) = Tracker.gradient(f, x, nest = true)[1]

df(2) # 14.0 (tracked)

# d²f/dx² = 6
d2f(x) = Tracker.gradient(df, x, nest = true)[1]

d2f(2) # 6.0 (tracked)

6.0 (tracked)

Gradienty możemy też obliczać [bez śledzenia](https://fluxml.ai/Flux.jl/stable/internals/tracker/#Taking-Gradients-1):

In [29]:
a, b = param(2), param(3)

c = a*b # 6.0 (tracked)

Tracker.back!(c)

In [30]:
Tracker.grad(a), Tracker.grad(b) # (3.0, 2.0)

(3.0, 2.0)

Dzięki tak zdefiniowanemu działaniu propagacji wstecznej Flux umożliwia samodzielne definiowanie algorytmu uczenia (uwaga! kod poniżej nie działa):

In [31]:
function simple_grad!(ps, η = .0001)
  for w in ps
    w.data .-= w.grad .* η
    w.grad .= 0
  end
end

simple_grad! (generic function with 2 methods)

albo wykorzystanie algorytmów wbudowanych w inne biblioteki Julii. Ponadto posiada zdefiniowane [podstawowe algorytmy optymalizacyjne:](https://fluxml.ai/Flux.jl/stable/training/optimisers/)

In [32]:
opt = ADAM(0.0001)

ADAM(0.0001, (0.9, 0.999), IdDict{Any,Any}())

### Uczenie modelu

Flux jest zdolny do kontrolowania całej procedury uczenia, nie musimy robić tego samodzielnie. Służy do tego funkcja <tt>train!</tt>:

In [33]:
Flux.train!(objective, data, opt)

UndefVarError: UndefVarError: objective not defined

Warto jednak zaznaczyć, że pozwala ona na uczenie jedynie przez pojedynczą epokę. Aby móc kontynuować proces uczenia dalej musimy w odpowiedni sposób przystować dane z których korzystamy:

In [38]:
using Base.Iterators: repeated
dataset = repeated((x, y), 200)

Base.Iterators.Take{Base.Iterators.Repeated{Tuple{Array{Float64,1},Array{Float64,1}}}}(Base.Iterators.Repeated{Tuple{Array{Float64,1},Array{Float64,1}}}(([0.770638, 0.417364, 0.957935, 0.105694, 0.47265], [0.84227, 0.676984])), 200)

albo skorzystać z makra <tt>@epochs</tt>:

In [35]:
Flux.@epochs 2 println("hello")

┌ Info: Epoch 1
└ @ Main C:\Users\p\.julia\packages\Flux\8XpDt\src\optimise\train.jl:107


hello
hello


┌ Info: Epoch 2
└ @ Main C:\Users\p\.julia\packages\Flux\8XpDt\src\optimise\train.jl:107


Pozwala ona też na definiowanie wywołań, które pozwolą nam kontrolować przebieg uczenia.

In [39]:
evalcb = () -> @show(loss(tX, tY))

#12 (generic function with 1 method)

## Przykład

In [40]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated

Zacznijmy od podstaw, wczytajmy i opracujmy zbiór danych na którym będziemy pracowali:

In [42]:

# Classify MNIST digits with a simple multi-layer-perceptron

imgs = MNIST.images()
# Stack images into one large batch
X = hcat(float.(reshape.(imgs, :))...) 

labels = MNIST.labels()
# One-hot-encode the labels
Y = onehotbatch(labels, 0:9) 

10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false   true  false  false  false  …  false  false  false  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false   true  false  false     false  false  false  false  false
  true  false  false  false  false  …  false  false   true  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false      true  false  false  false   true
 false  false  false  false   true     false  false  false  false  false

Zdefiniujmy model:

In [43]:
m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) 

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM()

Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))

accuracy(X, Y)

# Test set accuracy
tX = hcat(float.(reshape.(MNIST.images(:test), :))...) 
tY = onehotbatch(MNIST.labels(:test), 0:9) 

accuracy(tX, tY)

loss(X, Y) = 2.2464297f0 (tracked)
loss(X, Y) = 1.5402654f0 (tracked)
loss(X, Y) = 1.0656346f0 (tracked)
loss(X, Y) = 0.78270364f0 (tracked)
loss(X, Y) = 0.6237614f0 (tracked)
loss(X, Y) = 0.5302362f0 (tracked)
loss(X, Y) = 0.46996862f0 (tracked)
loss(X, Y) = 0.42835918f0 (tracked)
loss(X, Y) = 0.39791614f0 (tracked)
loss(X, Y) = 0.37264392f0 (tracked)
loss(X, Y) = 0.35132843f0 (tracked)
loss(X, Y) = 0.33548975f0 (tracked)
loss(X, Y) = 0.32227007f0 (tracked)
loss(X, Y) = 0.31098685f0 (tracked)
loss(X, Y) = 0.30114487f0 (tracked)
loss(X, Y) = 0.29239753f0 (tracked)
loss(X, Y) = 0.28451645f0 (tracked)
loss(X, Y) = 0.2784741f0 (tracked)


0.9239

### Strojenie hiperparametrów

Sieć neuronowa potrafi zoptymalizować jedynie wagi $\theta$ funkcji liniowych wykorzystanych do budowy modelu. Pozostałe parametry (funkcje aktywacji, metoda regularyzacji, stopa uczenia, etc.) muszą być przyjęte z góry. Dobrać je można na kilka różnych sposobów:
- wzorując się na literaturze
- zgadując parametry
- tworząc model [zdolny do nauczenia się optymalnej metody uczenia](https://github.com/FluxML/model-zoo/tree/master/other/meta-learning)
- przeszukując odpowiednio przestrzeń hiperparametrów

In [ ]:
using PyPlot

#### Krata równomierna

In [ ]:
p = hcat(sort(repeat(1:32,32)),repeat(1:32,32))
subplot(111, aspect="equal")
plot(p[:,1], p[:,2], "r.")

#### Liczby (pseudo)losowe

In [ ]:
p = rand(1024,2)
subplot(111, aspect="equal")
plot(p[:,1], p[:,2], "r.")

#### Sekwencje Sobola

In [ ]:
using Sobol
s = SobolSeq(2)
p = hcat([next!(s) for i = 1:1024]...)'
subplot(111, aspect="equal")
plot(p[:,1], p[:,2], "r.")

## Dodatkowa praca domowa

Dokonaj strojenia hiperparametrów sieci omawianej na wykładzie. Sprobuj znaleźć taką, która zapewnia wyższą trafność predykcji.